In [1]:
# 필요한 라이브러리 및 모듈

import numpy as np
import cv2
import butter
from matplotlib import pyplot as plt

In [2]:
# 1. Read Video file

# generate video object
filename = "./input/original_video_SR30.mp4"
cap = cv2.VideoCapture(filename)

In [3]:
# 2. Gray Scale로 변환

# 첫번째 프레임 읽기
ret, frame = cap.read()

# 영상의 사이즈 확인
frame_size = frame.shape
print("Shape of image is : ",frame.shape)

# Gray Sacle 이미지로 변환
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
print("Shape of image is : ",gray.shape)

Shape of image is :  (256, 640, 3)
Shape of image is :  (256, 640)


In [ ]:
# 3. Pyramid
# 한 프레임에 먼저 해보기

'''
Gau = [gray]
Down = cv2.pyrDown(gray)
Gau.append(Down)

for i in range (0, 5):
    Down = cv2.pyrDown(Down)
    Gau.append(Down)

Gau_copy = []
Gau_copy[:] = Gau
Gau_copy.reverse()

# 이미지를 sequential하게 저장
Gau_seq = [[],[],[],[],[],[]]
for i in range (0,6):
    Gau_seq[i].append(Gau_copy.pop())
    
Lap = []
for i in range (0, 6):
    Up = cv2.pyrUp(Gau.pop())
    Lap.append(Up - Gau[len(Gau)-1])
    
#Lap.reverse()

Lap_seq = [[],[],[],[],[],[]]
for i in range (0,6):
    Lap_seq[i].append(Lap.pop())'''

In [ ]:
# 모든 프레임에 대해 적용
'''
while ret:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #hist = cv2.equalizeHist(gray)
    
    Gau = [gray]
    Down = cv2.pyrDown(gray)
    Gau.append(Down)

    for i in range (0, 5):
        Down = cv2.pyrDown(Down)
        Gau.append(Down)
    
    Gau_copy = []
    Gau_copy[:] = Gau
    Gau_copy.reverse()
    
    for i in range (0,6):
        Gau_seq[i].append(Gau_copy.pop())

    Lap = []
    for i in range (0, 6):
        Up = cv2.pyrUp(Gau.pop())
        Lap.append(Up - Gau[len(Gau)-1])

    #Lap.reverse()
    
    for i in range (0,6):
        Lap_seq[i].append(Lap.pop())
    ret, frame = cap.read()'''

In [ ]:
# 3. 부동소수점 데이터로 먼저 변환
'''
for i in range(0, 3):
    for j in range(0, 101):
        Lap_seq[i][j] = Lap_seq[i][j].astype('float64')
        Lap_seq[i][j] = Lap_seq[i][j]/255'''

In [4]:
# 4. Pyramid
# pyramid 안쓰고, resize함수로 구현

def pyramidDown(input):
    # input is numpy array
    heigh, width = input.shape
    #output = np.zeros((size[0]//2, size[1]//2))
    blur = cv2.GaussianBlur(input, )
    output = cv2.resize(input, None, fx = 0.5, fy = 0.5)
    return output

def pyramidUp(input):
    # input is numpy array
    heigh, width = input.shape
    #output = np.zeros((size[0]//2, size[1]//2))
    output = cv2.resize(input, None, fx = 2, fy = 2, interpolation = cv2.INTER_CUBIC)
    return output

# 이미지의 빍기값을 0~1 사이로 정규화
gray = gray.astype('float64')
gray = gray / 255

Gau = [gray]
Down = pyramidDown(gray)

Gau.append(Down)

for i in range (0, 5):
    Down = pyramidDown(Down)
    Gau.append(Down)

Gau_copy = []
Gau_copy[:] = Gau
Gau_copy.reverse()

# 이미지를 sequential하게 저장
Gau_seq = [[],[],[],[],[],[]]
for i in range (0,6):
    Gau_seq[i].append(Gau_copy.pop())
    
Lap = []
for i in range (0, 6):
    Up = pyramidUp(Gau.pop())
    Lap.append(Up - Gau[len(Gau)-1])
    
#Lap.reverse()

Lap_seq = [[],[],[],[],[],[]]
for i in range (0,6):
    Lap_seq[i].append(Lap.pop())
    

In [5]:
# 모든 프레임에 대해 과정3 적용

while ret:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #hist = cv2.equalizeHist(gray)
    # 이미지의 빍기값을 0~1 사이로 정규화
    gray = gray.astype('float64')
    gray = gray / 255
    
    Gau = [gray]
    Down = pyramidDown(gray)
    Gau.append(Down)

    for i in range (0, 5):
        Down = pyramidDown(Down)
        Gau.append(Down)
    
    Gau_copy = []
    Gau_copy[:] = Gau
    Gau_copy.reverse()
    
    for i in range (0,6):
        Gau_seq[i].append(Gau_copy.pop())

    Lap = []
    for i in range (0, 6):
        Up = pyramidUp(Gau.pop())
        Lap.append(Up - Gau[len(Gau)-1])

    #Lap.reverse()
    
    for i in range (0,6):
        Lap_seq[i].append(Lap.pop())
    ret, frame = cap.read()

In [6]:
print(Lap_seq[0][0])
f = open('LaplacianPramid_L0_0_change1.csv', 'a')
np.savetxt(f, np.column_stack((Lap_seq[0][0])), delimiter=",", fmt='%s',  newline='\n')

[[ 0.00000000e+00 -2.22044605e-16  0.00000000e+00 ...  3.19896025e-04
   9.37069164e-05 -4.36221852e-05]
 [-1.11022302e-16  0.00000000e+00 -1.11022302e-16 ... -5.56586770e-04
  -6.58521465e-05  2.31574563e-04]
 [ 0.00000000e+00 -2.22044605e-16  0.00000000e+00 ...  1.96745629e-03
  -2.80851476e-04  7.33498966e-04]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -5.91297524e-03
  -3.09084724e-03 -1.37640261e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -3.82685942e-03
  -1.22870651e-03  3.59389361e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  7.53087062e-03
   2.10299024e-03  3.58572567e-03]]


In [7]:
# 4. 확대 주파수 결정 및 필터링
# Parameter Initializing 

SR = 2200 # SR is Sampling Rate
Fn = SR/2
T = 1/SR # T is Period
Low_freq = 350
High_freq = 370
alpha = 100.0

In [8]:
# 4.5 연산을 위해 0~1 사이의 부동소수점으로 변환
'''
for i in range(0, 3):
    for j in range(0, 101):
        Lap_seq[i][j] = Lap_seq[i][j].astype('float64')
        Lap_seq[i][j] = Lap_seq[i][j]/255'''

"\nfor i in range(0, 3):\n    for j in range(0, 101):\n        Lap_seq[i][j] = Lap_seq[i][j].astype('float64')\n        Lap_seq[i][j] = Lap_seq[i][j]/255"

In [9]:
# pyramid data 저장
'''
f = open('LaplacianPramid_L0_0.csv', 'a')

np.savetxt(f, np.column_stack((Lap_seq[0][0])), delimiter=",", fmt='%s',  newline='\n')'''

'\nf = open(\'LaplacianPramid_L0_0.csv\', \'a\')\n\nnp.savetxt(f, np.column_stack((Lap_seq[0][0])), delimiter=",", fmt=\'%s\',  newline=\'\n\')'

In [10]:
# 원하는 주파수 선택적으로 확대 
import time
start = time.time()
f=open('python_bandpass.csv','a')

# level 0
for i in range (0, frame_size[0]):
    for j in range (0, frame_size[1]):
        time_stack = []
        
        for k in range (0, 101):
            time_stack.append(Lap_seq[0][k][i][j])
            
        time_stack = butter.butter_bandpass_filter(time_stack, Low_freq, High_freq, SR, order=5)
        #np.savetxt(f, np.column_stack(time_stack), delimiter=",", fmt='%s',  newline='\n')
        for k in range (0, 101):
            Lap_seq[0][k][i][j] += alpha * time_stack[k]
            
end = time.time()
delay = end - start
print("1st level : ", delay)

1st level :  71.00811529159546


In [11]:
# level 1 
start = time.time()
for i in range (0, Lap_seq[1][0].shape[0]):
    for j in range (0, Lap_seq[1][0].shape[1]):
        time_stack = []
        
        for k in range (0, 101):
            time_stack.append(Lap_seq[1][k][i][j])
            
        time_stack = butter.butter_bandpass_filter(time_stack, Low_freq, High_freq, SR, order=5)
        
        for k in range (0, 101):
            Lap_seq[1][k][i][j] += alpha * time_stack[k]
            
end = time.time()
delay = end - start
print("2nd level : ", delay)

2nd level :  16.53279137611389


In [12]:
# level 2
start = time.time()

for i in range (0, Lap_seq[2][0].shape[0]):
    for j in range (0, Lap_seq[2][0].shape[1]):
        time_stack = []
        
        for k in range (0, 101):
            time_stack.append(Lap_seq[2][k][i][j])
            
        time_stack = butter.butter_bandpass_filter(time_stack, Low_freq, High_freq, SR, order=5)
        
        for k in range (0, 101):
            Lap_seq[2][k][i][j] += time_stack[k]
            
end = time.time()
delay = end - start
print("3nd level : ", delay)

3nd level :  4.170847177505493


In [13]:
# 변환했던 부동소수점 데이터를 다시 정수로 변환

for i in range(0, 3):
    for j in range(0, 101):
        Lap_seq[i][j] *= 255
        Lap_seq[i][j] = Lap_seq[i][j].astype('uint8')

for i in range(0, 3):
    for j in range(0, 101):
        Gau_seq[i][j] *= 255
        Gau_seq[i][j] = Gau_seq[i][j].astype('uint8')
        

In [14]:
# 5. 동영상으로 저장하기
# 영상 스택 쌓기

Rec2= []
for i in range (0, 101):
    Rec2.append( Lap_seq[2][i] + Gau_seq[2][i])

for i in range (0, 101):
    Rec2[i] = cv2.pyrUp(Rec2[i])

Rec1 = []
for i in range (0, 101):
    Rec1.append( Lap_seq[1][i] + Rec2[i] )

for i in range (0, 101):
    Rec1[i] = cv2.pyrUp(Rec1[i])

Rec0 = []
for i in range (0, 101):
    Rec0.append( Lap_seq[0][i] + Rec1[i] )

In [15]:
# Output 파일 생성 

size = Rec0[0].shape

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('./output/' + 'alpha_' + str(alpha) + 'output.mp4', fourcc, 30.0, (640, 256), isColor = False)

for i in range (0, len(Rec0)):
    #out.write(cv2.cvtColor(Rec0[i], cv2.COLOR_GRAY2RGB))
    out.write(Rec0[i])
#    cv2.imshow('frame', Rec0[i])
#    if cv2.waitKey(0) & 0xFF == ord('q'):
#            break

#cv2.destroyAllWindows()
            
out.release()

print("done")
print(size)

done
(256, 640)


In [16]:
cv2.imshow('ad', Lap_seq[1][1])
cv2.waitKey(0)
cv2.destroyAllWindows()